<a href="https://colab.research.google.com/github/Nikolay1982Nikolaev/Big-Data-Bicocca/blob/main/6_lexicon_based_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
filepath = '/content/drive/MyDrive/00.Last_sprint_UNIVERSITA/06.BigData/data'

### SA with Vader

In their official [repo](https://github.com/cjhutto/vaderSentiment) you can find the lexicon, a list of rules and the support ofr emoji recognition

In [3]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


In [4]:
sentences=["I like text mining so much, it is super fun",
           "I can't bear this lecture anymore",
           "today is quite sunny outside",
           "I hate flamingos, they're so inappropriate",
           "Catch utf-8 emoji such as 💘 and 💋 and 😁"]

In [5]:
!pip install vaderSentiment

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<50} {}".format(sentence, str(vs)))

I like text mining so much, it is super fun------- {'neg': 0.0, 'neu': 0.419, 'pos': 0.581, 'compound': 0.8658}
I can't bear this lecture anymore----------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
today is quite sunny outside---------------------- {'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}
I hate flamingos, they're so inappropriate-------- {'neg': 0.425, 'neu': 0.575, 'pos': 0.0, 'compound': -0.5719}
Catch utf-8 emoji such as 💘 and 💋 and 😁----------- {'neg': 0.0, 'neu': 0.583, 'pos': 0.417, 'compound': 0.875}


emoji [cheat sheet](https://unicode.org/emoji/charts/emoji-list.html)  (replace _U+_ with  _U000_ )

In [8]:
print("\U0001F923")


🤣


[here](https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/emoji_utf8_lexicon.txt) you can find the vader emoticons evaluations

### About the scoring

The Positive, Negative and Neutral scores represent the proportion of text that falls in these categories. This means our first sentence was rated as 58% Positive, 42% Neutral and 0% Negative. Hence all these should add up to 1.

The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

positive sentiment: compound score >= 0.05 <br>
neutral sentiment: (compound score > -0.05) and (compound score < 0.05) <br>
negative sentiment: compound score <= -0.05

__[Some rules](https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vaderSentiment.py):__  <br>


![picture](https://drive.google.com/uc?id=17TLK_0mwT3eB4Wzv9voPNfORJU-hD4Fw&authuser=lorenzo.malandri%40unimib.it&usp=drive_fs)



__Get words polarity:__
    
In the [lexicon](https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt) you can find, for each word in the vocab (7520 elements): <br>
Token, mean value, SD and scores of 10 independent human evaluators <br>

![picture](https://drive.google.com/uc?id=17SaF40CWbZSNdBjoUz5m8mLeaHpdtQq6&authuser=lorenzo.malandri%40unimib.it&usp=drive_fs)

### Polarity detection of Amazon data with Vader

In [16]:
import pandas as pd
#df = pd.read_csv(filepath+'/data/tripadvisor_binary.csv')
df = pd.read_csv("/content/drive/MyDrive/00.Last_sprint_UNIVERSITA/06.BigData/data/tripadvisor_binary.csv")
df.head()

,review,rating,bin_y
0,ok_nothing_special charge diamond_member hilto...,2,0
1,nice_room experience_hotel monaco seattle good...,3,-1
2,poor_value stayed monaco seattle july nice_hot...,2,0
3,nice_hotel nice_staff hotel_lovely staff_quite...,3,-1
4,horrible_customer service_hotel stay february ...,1,0


In [19]:
df = df[~df['review'].isna()]
df.Review = df.review.str.replace('_', ' ') #I do not have ngrams in the Vader lexicon

<ipython-input-19-32b8d533d2ca>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Review = df.review.str.replace('_', ' ') #I do not have ngrams in the Vader lexicon


In [22]:
# compute polarity scores
df['scores'] = df['review'].apply(lambda review: analyzer.polarity_scores(review))
df.head()

,review,rating,bin_y,scores
0,ok_nothing_special charge diamond_member hilto...,2,0,"{'neg': 0.037, 'neu': 0.794, 'pos': 0.169, 'co..."
1,nice_room experience_hotel monaco seattle good...,3,-1,"{'neg': 0.088, 'neu': 0.765, 'pos': 0.147, 'co..."
2,poor_value stayed monaco seattle july nice_hot...,2,0,"{'neg': 0.189, 'neu': 0.605, 'pos': 0.205, 'co..."
3,nice_hotel nice_staff hotel_lovely staff_quite...,3,-1,"{'neg': 0.15, 'neu': 0.732, 'pos': 0.118, 'com..."
4,horrible_customer service_hotel stay february ...,1,0,"{'neg': 0.106, 'neu': 0.743, 'pos': 0.151, 'co..."


Extract the compund value

In [23]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df.head()

,review,rating,bin_y,scores,compound
0,ok_nothing_special charge diamond_member hilto...,2,0,"{'neg': 0.037, 'neu': 0.794, 'pos': 0.169, 'co...",0.9804
1,nice_room experience_hotel monaco seattle good...,3,-1,"{'neg': 0.088, 'neu': 0.765, 'pos': 0.147, 'co...",0.9169
2,poor_value stayed monaco seattle july nice_hot...,2,0,"{'neg': 0.189, 'neu': 0.605, 'pos': 0.205, 'co...",0.2023
3,nice_hotel nice_staff hotel_lovely staff_quite...,3,-1,"{'neg': 0.15, 'neu': 0.732, 'pos': 0.118, 'com...",-0.4588
4,horrible_customer service_hotel stay february ...,1,0,"{'neg': 0.106, 'neu': 0.743, 'pos': 0.151, 'co...",0.8360


compute binary predictions

In [24]:
df['predicted'] = df['compound'].apply(lambda c: 1 if c >=0 else 0)
df.head()

,review,rating,bin_y,scores,compound,predicted
0,ok_nothing_special charge diamond_member hilto...,2,0,"{'neg': 0.037, 'neu': 0.794, 'pos': 0.169, 'co...",0.9804,1
1,nice_room experience_hotel monaco seattle good...,3,-1,"{'neg': 0.088, 'neu': 0.765, 'pos': 0.147, 'co...",0.9169,1
2,poor_value stayed monaco seattle july nice_hot...,2,0,"{'neg': 0.189, 'neu': 0.605, 'pos': 0.205, 'co...",0.2023,1
3,nice_hotel nice_staff hotel_lovely staff_quite...,3,-1,"{'neg': 0.15, 'neu': 0.732, 'pos': 0.118, 'com...",-0.4588,0
4,horrible_customer service_hotel stay february ...,1,0,"{'neg': 0.106, 'neu': 0.743, 'pos': 0.151, 'co...",0.8360,1


In [25]:
from sklearn.metrics import classification_report

print(classification_report(df.bin_y, df.predicted))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      2184
           0       0.78      0.42      0.54      3214
           1       0.00      0.00      0.00         0

    accuracy                           0.25      5398
   macro avg       0.26      0.14      0.18      5398
weighted avg       0.47      0.25      0.32      5398



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.